In [2]:
# Installationsbefehle (normalerweise nur einmal pro Session ausführen)
!pip install -q xlrd
!git clone https://github.com/Lossophy/BINA-Projekt.git # Nur einmal ausführen, um das Repo zu klonen
!pip install panel --quiet
!pip install jupyter_bokeh --quiet # Bokeh ist eine Abhängigkeit von Panel für Jupyter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # Behalten wir für den Fall, dass es für andere Teile gebraucht wird
import seaborn as sns # Behalten wir für den Fall
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets # Momentan nicht aktiv für F3, aber für andere Teile evtl.
from IPython.display import display, clear_output # Momentan nicht aktiv für F3
import panel as pn
pn.extension('plotly',sizing_mode="stretch_width") # sizing_mode global setzen für alle Plotly Panes

Cloning into 'BINA-Projekt'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 236 (delta 41), reused 4 (delta 2), pack-reused 165 (from 1)
Receiving objects: 100% (236/236), 5.16 MiB | 9.04 MiB/s, done.
Resolving deltas: 100% (130/130), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.3 MB/s eta 0:00:00


In [3]:
# Datei einlesen
df = pd.read_csv('/content/BINA-Projekt/Data/ogd115_gest_bilanz.csv')

# Daten überprüfen
print(df.head())

   Jahr           Rubrik Energietraeger        TJ
0  1980  Bruttoverbrauch   Elektrizität  -29450.0
1  1980  Bruttoverbrauch  Erdölprodukte  321250.0
2  1980  Bruttoverbrauch      Fernwärme       0.0
3  1980  Bruttoverbrauch            Gas   36280.0
4  1980  Bruttoverbrauch    Holzenergie   26280.0


In [21]:
# Harmlose Bokeh-Warnmeldung ausblenden
import warnings
from bokeh.util.warnings import BokehUserWarning
warnings.filterwarnings("ignore", category=BokehUserWarning)

# Grundlegende Datenaufbereitung
df['TJ'] = pd.to_numeric(df['TJ'], errors='coerce').fillna(0)
df['Jahr'] = df['Jahr'].astype(int)

# --- Definitionen für F3 ---
erneuerbare_inland_basis_f3 = ["Holzenergie", "Wasserkraft", "Uebrige erneuerbare Energien"]
muell_energietraeger_f3 = "Müll und Industrieabfälle" # Einzelner String für leichtere Prüfung
# erneuerbare_inland_mit_muell_f3 wird jetzt in prepare_f3_data dynamischer gehandhabt
fossile_import_traeger_f3 = ["Erdölprodukte", "Gas", "Kohle", "Rohöl"]

# Farbcodierung für F3
farben_erneuerbar_inland_f3 = {
    "Holzenergie": "#1f77b4", "Wasserkraft": "#aec7e8", "Uebrige erneuerbare Energien": "#72bcd4",
    "Müll und Industrieabfälle (50% erneuerbar)": "#2ca02c" # Angepasster Name für Legende
}
farben_fossil_import_f3 = {
    "Erdölprodukte": "#ff7f0e", "Gas": "#ffbb78", "Kohle": "#d62728", "Rohöl": "#ff9896"
}

# color_map_f3_szenario1_detail bleibt wie es ist
color_map_f3_szenario1_detail = {
    **{k: v for k, v in farben_erneuerbar_inland_f3.items() if k != "Müll und Industrieabfälle (50% erneuerbar)"},
    **farben_fossil_import_f3
}

# color_map_f3_szenario2_detail wird angepasst, um den neuen Namen für Müll zu verwenden
color_map_f3_szenario2_detail = {**farben_erneuerbar_inland_f3, **farben_fossil_import_f3}


# --- Hilfsfunktion zur Datenaggregation (angepasst für 50% Müll) ---
def prepare_f3_data(df_input, erneuerbare_liste_basis, fossile_liste, muell_anteil_prozent=0, szenario_label=""):
    df_erneuerbare_inland = df_input[
        (df_input['Rubrik'] == "Inlandproduktion") &
        (df_input['Energietraeger'].isin(erneuerbare_liste_basis))
    ].copy() # .copy() um SettingWithCopyWarning zu vermeiden

    # Behandlung von Müll und Industrieabfälle
    df_muell_inland = pd.DataFrame() # Initialisieren für den Fall, dass muell_anteil_prozent = 0
    if muell_anteil_prozent > 0:
        df_muell_temp = df_input[
            (df_input['Rubrik'] == "Inlandproduktion") &
            (df_input['Energietraeger'] == muell_energietraeger_f3)
        ].copy()
        df_muell_temp['TJ'] = df_muell_temp['TJ'] * (muell_anteil_prozent / 100.0)
        df_muell_temp['Energietraeger'] = f"{muell_energietraeger_f3} ({muell_anteil_prozent}% erneuerbar)" # Für Legende
        df_muell_inland = df_muell_temp

    # Kombinieren der Basis-Erneuerbaren mit dem Müll-Anteil
    df_ern_inland_detail_final = pd.concat([df_erneuerbare_inland, df_muell_inland])

    # Detail-Daten für Plotting vorbereiten
    df_ern_inland_detail_grouped = df_ern_inland_detail_final.groupby(['Jahr', 'Energietraeger'])['TJ'].sum().reset_index()
    df_ern_inland_detail_grouped['Hauptkategorie'] = f'Inländ. Erneuerbare {szenario_label}'.strip()


    ern_inland_summe = df_ern_inland_detail_final.groupby('Jahr')['TJ'].sum().reset_index()
    ern_inland_summe = ern_inland_summe.rename(columns={'TJ': 'Erneuerbare_Inland_TJ'})

    df_foss_import_detail = df_input[
        (df_input['Rubrik'] == "Import") &
        (df_input['Energietraeger'].isin(fossile_liste))
    ].groupby(['Jahr', 'Energietraeger'])['TJ'].sum().reset_index()
    df_foss_import_detail['Hauptkategorie'] = 'Importe fossiler Energieträger'

    foss_import_summe = df_foss_import_detail.groupby('Jahr')['TJ'].sum().reset_index()
    foss_import_summe = foss_import_summe.rename(columns={'TJ': 'Fossile_Import_TJ'})

    df_detail_kombiniert_final = pd.concat([df_ern_inland_detail_grouped, df_foss_import_detail])

    df_vergleich = pd.merge(ern_inland_summe, foss_import_summe, on='Jahr', how='outer').fillna(0)
    df_vergleich = df_vergleich.sort_values(by='Jahr').reset_index(drop=True)

    df_vergleich['Verhaeltnis_ErnIn_vs_FossImp'] = df_vergleich['Erneuerbare_Inland_TJ'] / df_vergleich['Fossile_Import_TJ'].replace(0, np.nan)
    df_vergleich['Verhaeltnis_ErnIn_vs_FossImp'] = df_vergleich['Verhaeltnis_ErnIn_vs_FossImp'].fillna(0)

    summe_energien = df_vergleich['Erneuerbare_Inland_TJ'] + df_vergleich['Fossile_Import_TJ']
    label_suffix = f" {szenario_label}".rstrip()
    df_vergleich[f'Anteil_Erneuerbare_Inland_Prozent{label_suffix}'] = (df_vergleich['Erneuerbare_Inland_TJ'] / summe_energien.replace(0, np.nan) * 100).fillna(0)
    df_vergleich[f'Anteil_Fossile_Import_Prozent{label_suffix}'] = (df_vergleich['Fossile_Import_TJ'] / summe_energien.replace(0, np.nan) * 100).fillna(0)

    return df_detail_kombiniert_final, df_vergleich

# --- Daten für Szenarien vorbereiten ---
# Szenario 1: Basis Erneuerbare (ohne Müll)
df_detail_s1_f3, df_vergleich_s1_f3 = prepare_f3_data(df, erneuerbare_inland_basis_f3, fossile_import_traeger_f3, muell_anteil_prozent=0, szenario_label="(Basis)")

# Szenario 2: Erneuerbare INKLUSIVE 50% von Müll und Industrieabfälle
df_detail_s2_f3, df_vergleich_s2_f3 = prepare_f3_data(df, erneuerbare_inland_basis_f3, fossile_import_traeger_f3, muell_anteil_prozent=50, szenario_label="(inkl. 50% Müll)")

In [34]:
def plot_f3_s2_absolut_aggregiert_tab():
    if df_vergleich_s2_f3.empty: return go.Figure().update_layout(title_text="Daten für Szenario 2 nicht geladen") # Geändert auf df_vergleich_s2_f3
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_vergleich_s2_f3['Jahr'], y=df_vergleich_s2_f3['Erneuerbare_Inland_TJ'],
                             mode='lines+markers', name='Inländ. Erneuerbare (inkl. 50% Müll)', line=dict(color=farben_erneuerbar_inland_f3["Müll und Industrieabfälle (50% erneuerbar)"]))) # Legendenname und Farbe angepasst
    fig.add_trace(go.Scatter(x=df_vergleich_s2_f3['Jahr'], y=df_vergleich_s2_f3['Fossile_Import_TJ'],
                             mode='lines+markers', name='Importe fossiler Energieträger', line=dict(color=farben_fossil_import_f3["Erdölprodukte"])))

    fig.update_layout(
        title_text="Abs. Mengen (aggregiert): Erneuerbare (inkl. 50% Müll) vs. Fossile Importe", # Titel angepasst
        title_x=0.5,
        xaxis_title="Jahr",
        yaxis_title="Energiemenge (TJ)",
        hovermode="x unified",
        legend_title_text='Kategorie',
        legend=dict(traceorder="reversed"))
    return fig

In [35]:
def plot_f3_s2_absolut_detail_tab():
    if df_detail_s2_f3.empty: return go.Figure().update_layout(title_text="Daten für Szenario 2 nicht geladen") # Geändert auf df_detail_s2_f3
    # Erstelle eine benutzerdefinierte Sortierreihenfolge für die Legende und Stapelung
    kategorie_reihenfolge_detail_s2 = [et for et in erneuerbare_inland_basis_f3]
    kategorie_reihenfolge_detail_s2.append("Müll und Industrieabfälle (50% erneuerbar)") # Angepasster Name
    kategorie_reihenfolge_detail_s2.extend(fossile_import_traeger_f3)

    fig = px.area(df_detail_s2_f3, x="Jahr", y="TJ", color="Energietraeger", line_group="Hauptkategorie", # line_group beibehalten oder anpassen, falls nötig
                  color_discrete_map=color_map_f3_szenario2_detail,
                  title="Abs. Mengen (detailliert): Erneuerbare (inkl. 50% Müll) vs. Fossile Importe", # Titel angepasst
                  labels={"TJ": "Energiemenge (TJ)", "Energietraeger": "Energieträger"},
                  category_orders={"Energietraeger": kategorie_reihenfolge_detail_s2})

    fig.update_layout(
        title_x=0.5,
        legend_title_text='Energieträger',
        hovermode="x unified",
        legend=dict(traceorder="reversed"))
    return fig

In [36]:
def plot_f3_s2_anteil_100prozent_aggregiert_tab():
    if df_vergleich_s2_f3.empty: return go.Figure().update_layout(title_text="Daten für Szenario 2 nicht geladen")
    df_anteile_melted = df_vergleich_s2_f3.melt(id_vars=['Jahr'],
                                              value_vars=['Anteil_Erneuerbare_Inland_Prozent (inkl. 50% Müll)', 'Anteil_Fossile_Import_Prozent (inkl. 50% Müll)'], # Angepasste Spaltennamen
                                              var_name='Kategorie_Anteil', value_name='Prozentualer_Anteil')
    df_anteile_melted['Kategorie_Anteil'] = df_anteile_melted['Kategorie_Anteil'].replace({
        'Anteil_Erneuerbare_Inland_Prozent (inkl. 50% Müll)': 'Anteil Inländ. Erneuerbare (inkl. 50% Müll)', # Angepasst
        'Anteil_Fossile_Import_Prozent (inkl. 50% Müll)': 'Anteil Fossile Importe'
    })
    fig = px.area(df_anteile_melted, x="Jahr", y="Prozentualer_Anteil", color="Kategorie_Anteil",
                  title="Anteilige Entwicklung (aggregiert, 100% Ansicht, inkl. 50% Müll)", # Titel angepasst
                  labels={"Prozentualer_Anteil": "Prozentualer Anteil (%)"},
                  color_discrete_map={'Anteil Inländ. Erneuerbare (inkl. 50% Müll)': farben_erneuerbar_inland_f3["Müll und Industrieabfälle (50% erneuerbar)"], 'Anteil Fossile Importe': '#ff7f0e'}, # Farbe angepasst
                  groupnorm='percent')

    fig.update_layout(
        title_x=0.5,
        legend_title_text='Energiekategorie',
        hovermode="x unified",
        legend=dict(traceorder="reversed"))
    fig.update_yaxes(ticksuffix="%")
    return fig

In [37]:
def plot_f3_s2_verhaeltnis_tab():
    if df_vergleich_s2_f3.empty: return go.Figure().update_layout(title_text="Daten für Szenario 2 nicht geladen") # Geändert auf df_vergleich_s2_f3
    fig = px.line(df_vergleich_s2_f3, x="Jahr", y="Verhaeltnis_ErnIn_vs_FossImp",
                  title="Verhältnis Inl. Erneuerbare (inkl. 50% Müll) zu Fossilen Importen", # Titel angepasst
                  labels={"Verhaeltnis_ErnIn_vs_FossImp": "Verhältnis"}, markers=True,
                  color_discrete_sequence=[farben_erneuerbar_inland_f3["Müll und Industrieabfälle (50% erneuerbar)"]]) # Farbe hinzugefügt
    fig.add_hline(y=1, line_dash="dash", line_color="grey", annotation_text="Gleichstand")

    fig.update_layout(
        title_x=0.5,
        hovermode="x unified",
        legend=dict(traceorder="reversed"))
    return fig

In [38]:
def plot_f3_s2_anteil_summe_tab():
    if df_vergleich_s2_f3.empty:
        return go.Figure().update_layout(title_text="Daten für Szenario 2 nicht geladen")

    fig = px.line(df_vergleich_s2_f3,
                  x='Jahr',
                  y='Anteil_Erneuerbare_Inland_Prozent (inkl. 50% Müll)', # Angepasster Spaltenname
                  title='Anteil Inl. Erneuerbare (inkl. 50% Müll) an (Inl. Erneuerbare (inkl. 50% Müll) + Fossile Importe)', # Titel angepasst
                  labels={'Anteil_Erneuerbare_Inland_Prozent (inkl. 50% Müll)': 'Anteil (%)'}, # Label angepasst
                  markers=True,
                  line_shape='linear',
                  color_discrete_sequence=[farben_erneuerbar_inland_f3["Müll und Industrieabfälle (50% erneuerbar)"]]) # Passende Farbe

    fig.update_layout(
        title_x=0.5,
        xaxis_title="Jahr",
        yaxis_title="Anteil (%)",
        hovermode="x unified",
        yaxis_ticksuffix="%",
        legend=dict(traceorder="reversed")
    )
    return fig

In [39]:
def plot_f3_s2_anteil_100prozent_detail_tab():
    if df_detail_s2_f3.empty:
        return go.Figure().update_layout(title_text="Daten für Szenario 2 (inkl. 50% Müll) nicht geladen") # Titel angepasst

    df_detail_s2_f3_sorted = df_detail_s2_f3.copy()
    # Erstelle eine benutzerdefinierte Sortierreihenfolge für die Legende und Stapelung
    kategorie_reihenfolge_detail_s2 = [et for et in erneuerbare_inland_basis_f3]
    kategorie_reihenfolge_detail_s2.append("Müll und Industrieabfälle (50% erneuerbar)") # Angepasster Name
    kategorie_reihenfolge_detail_s2.extend(fossile_import_traeger_f3)

    df_detail_s2_f3_sorted['Energietraeger'] = pd.Categorical(
        df_detail_s2_f3_sorted['Energietraeger'],
        categories=kategorie_reihenfolge_detail_s2,
        ordered=True
    )
    df_detail_s2_f3_sorted = df_detail_s2_f3_sorted.sort_values(by=['Jahr', 'Energietraeger'])

    fig = px.area(df_detail_s2_f3_sorted,
                  x="Jahr",
                  y="TJ",
                  color="Energietraeger",
                  line_group="Energietraeger",
                  groupnorm='percent',
                  title="Anteilige Entwicklung (detailliert, 100% Ansicht): Erneuerbare (inkl. 50% Müll) vs. Fossile Importe", # Titel angepasst
                  labels={"TJ": "Prozentualer Anteil (%)", "Energietraeger": "Energieträger"},
                  color_discrete_map=color_map_f3_szenario2_detail, # Verwendet die aktualisierte color_map
                  category_orders={"Energietraeger": kategorie_reihenfolge_detail_s2}
                 )

    fig.update_layout(
        title_x=0.5,
        legend_title_text='Energieträger',
        hovermode="x unified",
        yaxis_ticksuffix="%",
        legend=dict(traceorder="reversed")
    )
    return fig

In [42]:
# --- Module installieren und Panel-Dashboard erstellen ---
dashboard_f3 = pn.Tabs(
    ("G1", pn.pane.Plotly(plot_f3_s2_absolut_aggregiert_tab())),
    ("G2", pn.pane.Plotly(plot_f3_s2_absolut_detail_tab())),
    ("G3", pn.pane.Plotly(plot_f3_s2_anteil_100prozent_aggregiert_tab())),
    ("G4", pn.pane.Plotly(plot_f3_s2_verhaeltnis_tab())),
    ("G5", pn.pane.Plotly(plot_f3_s2_anteil_summe_tab())),
    ("G6", pn.pane.Plotly(plot_f3_s2_anteil_100prozent_detail_tab())),


    dynamic=True # Lädt Tabs bei Bedarf, kann bei vielen Tabs performanter sein
)

# --- Anzeige des Dashboards ---
dashboard_f3.servable(title="F3: Analyse Erneuerbare Inland vs. Fossile Importe")

Tabs(dynamic=True, sizing_mode='stretch_width')
    [0] Plotly(Figure, sizing_mode='stretch_width')
    [1] Plotly(Figure, sizing_mode='stretch_width')
    [2] Plotly(Figure, sizing_mode='stretch_width')
    [3] Plotly(Figure, sizing_mode='stretch_width')
    [4] Plotly(Figure, sizing_mode='stretch_width')
    [5] Plotly(Figure, sizing_mode='stretch_width')